# Let's take a look at the General Service List (list of common english words) ... 2000 words or so

https://www.eapfoundation.com/vocab/general/gsl/

# Corpus of Contemporary American English (COCA) <-- sorted by frequency 
https://www.wordfrequency.info/ 
UGH i have to purchase this? booo

# Game plan...
* I want certain reactive groups to correlate with whether things are nouns or verbs.
    * I think that means I have to somehow separate out reactive vs. unreactive chemicals first.
    * Similarly separate out verbs and nouns in english (that should have a function already)
* Then I want to maybe match the most common words in each category using COCA for english and some other sort of frequency of most common chemicals...
* After frequency, let's rely on embeddings to get everything else closest to the nearest neighbor anchor point. This way we can be sure things are at least somewhat aligned.
    * alternatively we don't even start with the anchors but jump straight into the embeddings? Need to talk to David to get his help here with alignment.


* Subjunctive / conditional / other fun tenses could be inert gases that I add to the reaction like Helium could be a good example of conditional.
* Electro chemistry would be crazy... maybe that can be for a tense that is crazy... again I want to say conditional.

* Past, present, future will be decided by which side of a reaction is given. if it's in equilibrium then it's present.

I guess stoichometry is important here :^P

* One thing I wish would be that we have 

In [24]:
import nltk
nltk.download('brown')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package brown to
[nltk_data]     /Users/yitongtseo/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/yitongtseo/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/yitongtseo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/yitongtseo/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [34]:
# POS tag descriptions based on the Penn Treebank tag set
pos_tag_descriptions = {
    "CC": "Coordinating conjunction",
    "CD": "Cardinal number",
    "DT": "Determiner",
    "EX": "Existential there",
    "FW": "Foreign word",
    "IN": "Preposition or subordinating conjunction",
    "JJ": "Adjective",
    "JJR": "Adjective",
    "JJS": "Adjective",
    "LS": "List item marker",
    "MD": "Modal",
    "NN": "Noun",
    "NNS": "Noun",
    "NNP": "Noun", # Proper Noun
    "NNPS": "Noun",
    "PDT": "Predeterminer",
    "POS": "Possessive ending",
    "PRP": "Noun", # Personal pronoun
    "PRP$": "Noun", # Possessive pronoun
    "RB": "Adverb",
    "RBR": "Adverb",
    "RBS": "Adverb",
    "RP": "Particle",
    "SYM": "Symbol",
    "TO": "to",
    "UH": "Interjection",
    "VB": "Verb",
    "VBD": "Verb",
    "VBG": "Verb",
    "VBN": "Verb",
    "VBP": "Verb",
    "VBZ": "Verb",
    "WDT": "Wh-determiner",
    "WP": "Wh-pronoun",
    "WP$": "Possessive wh-pronoun",
    "WRB": "Wh-adverb",
    "``": "Punctuation",
    "''": "Punctuation",
    ",": "Punctuation",
    ".": "Punctuation",
    ":": "Punctuation",
    "(": "Punctuation",
    ")": "Punctuation",
    "$": "Punctuation",
    "#": "Punctuation",
}

In [26]:
from nltk.corpus import brown
from nltk import FreqDist, pos_tag
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
import re

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to determine the lemma based on POS tag
def get_lemma(word, tag):
    # Exclude words that are entirely numeric or symbols
    if re.fullmatch(r'[\d$.,]+', word):
        return None  # Return None to indicate exclusion
    
    # Handle possessive cases
    if word.endswith("'s") or word.endswith("’s"):
        word = word[:-2]  # Remove 's
    elif word.endswith("'") or word.endswith("’"):
        word = word[:-1]  # Remove trailing apostrophe

    if tag.startswith('NN'):
        return lemmatizer.lemmatize(word, pos='n')  # for nouns
    elif tag.startswith('VB'):
        return lemmatizer.lemmatize(word, pos='v')  # for verbs
    elif tag.startswith('JJ'):
        return lemmatizer.lemmatize(word, pos='a')  # for adjectives
    elif tag.startswith('RB'):
        return lemmatizer.lemmatize(word, pos='r')  # for adverbs
    else:
        return word
    

words = [word.lower() for word in brown.words()]
tagged_words = pos_tag(words)

In [42]:
tagged_lemmas = [
    (get_lemma(word, pos), pos_tag_descriptions[pos]) for (word, pos) in tagged_words
]
total_freq_dist = FreqDist(tagged_lemmas)
sorted_words_by_POS = {}
for (lemma, pos), count in total_freq_dist.items():
    sorted_words_by_POS[pos] = sorted_words_by_POS.get(pos, []) + [(lemma, count)]

for pos, word_list in sorted_words_by_POS.items():
    sorted_words_by_POS[pos] = sorted(word_list, key=lambda item: item[1], reverse=True)


In [46]:
len(sorted_words_by_POS['Noun'])

24469

### Now let's get their embeddings and roughly cluster the top 1000 or so words

In [ ]:
from transformers import BertModel, BertTokenizer
import torch

# Initialize tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")


def get_word_embeddings(words):
    inputs = tokenizer(words, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)

    embeddings = outputs.last_hidden_state[:, 0, :]
    return embeddings

In [ ]:
from dataclasses import dataclass

@dataclass
class Word:
    pos: str
    word: str
    count: int

    def get_embedding(self) -> float:
        return get_word_embeddings([self.word])[0]


# sorted_words_embeddings_by_POS = {}

# for pos, word_list in sorted_words_by_POS.items():
#     sorted_words_by_POS[pos] = sorted(word_list, key=lambda item: item[1], reverse=True)


# The rest is easy enough... what about getting the top molecules now?

Okay let's take a swing at Enamine Building Blocks, I found a list of 500K of them. that should be a good start.
Can we create their morgan fingerprints and cluster them?

Taken from https://zinc12.docking.org/db/byvendor/enaminebb/enaminebb.in


In [5]:
import pandas as pd

enamine_df = pd.read_csv('enaminebb.in.txt', delimiter=' ', index_col=False)

In [7]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from tqdm import tqdm


def get_single_molecule_embedding(smiles, radius=5, nBits=2048):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=radius, nBits=nBits)
        return list(fp)
    return None


def process_and_append(
    df, start_idx=0, chunk_size=1000, output_file="final_processed_data.csv"
):
    for start in tqdm(range(start_idx, len(df), chunk_size)):
        end = start + chunk_size
        chunk = df.iloc[start:end].copy()  # Using copy to safely modify the data
        chunk["Morgan_fingerprint"] = chunk["SMILES"].apply(
            get_single_molecule_embedding
        )
        mode = (
            "a" if start > 0 else "w"
        )  # 'w' for write on first chunk, 'a' for append on subsequent chunks
        header = True if start == 0 else False  # Write header only for the first chunk
        chunk.to_csv(output_file, mode=mode, header=header, index=False)


def check_progress(output_file):
    """Check if the output file already exists and determine the last processed index"""
    try:
        last_part = pd.read_pickle(output_file)
        last_index_processed = last_part.index[-1]
        return last_index_processed + 1
    except FileNotFoundError:
        return 0


# Check for existing progress and determine where to resume
output_file = "processed_data.pkl"
start_index = check_progress(output_file)

# Process DataFrame from the last checkpoint
process_and_append(
    enamine_df, start_idx=start_index, chunk_size=1000, output_file=output_file
)

100%|██████████| 522/522 [13:58<00:00,  1.61s/it]


In [6]:
import pandas as pd
output_file = "processed_data.pkl"
enamine_df = pd.read_pickle(output_file)

UnpicklingError: the STRING opcode argument must be quoted

In [ ]:
len(enamine_df)

1000

We will use AIZynthFinder for the retrosynthesis planning

https://github.com/MolecularAI/aizynthfinder

https://molecularai.github.io/aizynthfinder/

In [2]:
from aizynthfinder.interfaces import AiZynthApp
app = AiZynthApp("config.yml")


Loading template-based expansion policy model from /Users/yitongtseo/Documents/GitHub/ethical_necromancy/Chemical_Language(Glucose for now)/uspto_model.onnx to uspto
Loading templates from /Users/yitongtseo/Documents/GitHub/ethical_necromancy/Chemical_Language(Glucose for now)/uspto_templates.csv.gz to uspto
Loading template-based expansion policy model from /Users/yitongtseo/Documents/GitHub/ethical_necromancy/Chemical_Language(Glucose for now)/uspto_ringbreaker_model.onnx to ringbreaker
Loading templates from /Users/yitongtseo/Documents/GitHub/ethical_necromancy/Chemical_Language(Glucose for now)/uspto_ringbreaker_templates.csv.gz to ringbreaker
Loading filter policy model from /Users/yitongtseo/Documents/GitHub/ethical_necromancy/Chemical_Language(Glucose for now)/uspto_filter_model.onnx to uspto
Loading stock from InMemoryInchiKeyQuery to zinc


Text(value='', continuous_update=False, description='SMILES')

Output(layout=Layout(border='1px solid silver', height='180px', width='50%'))

Output(layout=Layout(border='1px solid silver', height='320px', overflow='auto', width='99%'))

Output(layout=Layout(border='1px solid silver', width='99%'))

In [4]:
!aizynthcli --config config.yml --smiles "COc1cccc(OC(=O)/C=C/c2cc(OC)c(OC)c(OC)c2)c1"

Loading template-based expansion policy model from /Users/yitongtseo/Documents/GitHub/ethical_necromancy/Chemical_Language(Glucose for now)/uspto_model.onnx to uspto
Loading templates from /Users/yitongtseo/Documents/GitHub/ethical_necromancy/Chemical_Language(Glucose for now)/uspto_templates.csv.gz to uspto
Loading template-based expansion policy model from /Users/yitongtseo/Documents/GitHub/ethical_necromancy/Chemical_Language(Glucose for now)/uspto_ringbreaker_model.onnx to ringbreaker
Loading templates from /Users/yitongtseo/Documents/GitHub/ethical_necromancy/Chemical_Language(Glucose for now)/uspto_ringbreaker_templates.csv.gz to ringbreaker
Loading filter policy model from /Users/yitongtseo/Documents/GitHub/ethical_necromancy/Chemical_Language(Glucose for now)/uspto_filter_model.onnx to uspto
Loading stock from InMemoryInchiKeyQuery to zinc
Selected as stock: zinc
Compounds in stock: 17422831
Selected as expansion policy: uspto
Selected as filter policy: uspto
Trees saved to tre

In [ ]:
from tqdm import tqdm
import pandas as pd

# Assuming enamine_df is your DataFrame and it has a column 'SMILES'
# Applying the fingerprint function to each SMILES entry
tqdm.pandas(desc="Generating fingerprints")
enamine_df["Morgan_fingerprint"] = enamine_df["SMILES"].progress_apply(
    get_single_molecule_embedding
)

In [79]:
# import tqdm as tqdm

# # TODO: add a column to enamine_df that is called "Morgan_fingerprint"
# for idx, row in tqdm(enamine_df.iterrows(), total=len(enamine_df)):
#     enamine_df.iloc[idx]["Morgan Fingerprint"] = get_molecule_embeddings(
#         [row["SMILES"]]
#     )[0]

[10, 6, 4, 39, 44, 97, 38]

In [71]:

# # # Example usage
# smiles_list = [
#     "C(C(=O)O)N",
#     "CCO",
#     "C1=CC=CC=C1",
#     "COc1cc2ccccc2cc1C(=O)O",
#     "c1ccc2c(c1)C(=O)c3cccc(c3C2=O)C(=O)O",
#     "c1cc(cc(c1)Oc2cccc(c2)C(F)(F)F)C3N(C(=O)CS3)CCN4CCCCC4",
#     "COC(=O)c1cccc(c1S(=O)(=O)F)N"
# ]  # Glycine, Ethanol, Benzene
# fingerprints = get_molecule_embeddings(smiles_list)
# # print(fingerprints)
# # print(fingerprints[0].sum())
# # print(fingerprints[1].sum())
# [fp.sum() for fp in fingerprints]

import numpy as np
# print(np.array(fingerprints[2]).where(1))
numpy_arr = np.array(fingerprints[2])
[idx for idx, val  in enumerate(numpy_arr) if val > 0]
fingerprints[2][389]

1

## Taking a step back... nouns can be just described by their constitutive atoms? But complex nouns then get out of grasp... like how would you chemically describe a human?

In [ ]:
import csv

# Writing to a CSV file
with open('most_common_words.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['Word', 'Frequency'])  # Writing header
    writer.writerows(most_common_words)


# PubChem to the rescue

FTP to download all of the compounds in pubchem as a SMILES -> ftp://ftp.ncbi.nlm.nih.gov/pubchem/Compound/Extras/CID-SMILES.gz
https://chemistry.stackexchange.com/questions/122109/how-to-get-the-smiles-of-all-compounds-on-pubchem 

Hopefully it's only a couple of gigabytes

Then to get the 1000 most common molecules is a bit difficult. but perhpas something I can do is just cheat:
* Either take the 1000 cheapest molecules
* Or take the 1000 molecules with the shortest SMILE strings (That ensures I get all of the elements, H2O, methane, etc.)

Maybe not necessary, but still may help is to bring them both down to the same dimension size with PCA
This step can help in normalizing the scale and variance of the datasets.
Especially if I bring them both lower... maybe 100?

* Canonical Correlation Analysis (CCA): If you seek more sophisticated statistical techniques, CCA can be used to find the relationships between two sets of variables. This might give insight`s into how molecular structures correlate with semantic meanings of words.
* Procrustes Analysis: This method is used to find the optimal alignment of two datasets by minimizing the distance between them after allowing for translation, scaling, and rotation. It's more complex but can effectively align two spaces

In [ ]:
from scipy.spatial import procrustes

# Assume matrix1 and matrix2 are your datasets (numpy arrays)
# They should be of the same dimensions

# Apply Procrustes Analysis
mtx1_transformed, mtx2_transformed, disparity = procrustes(matrix1, matrix2)

# mtx1_transformed and mtx2_transformed are the transformed matrices
# 'disparity' measures the sum of the squared differences (lower is better fit)


In [ ]:
from sklearn.cross_decomposition import CCA
import numpy as np

# Assume X and Y are your datasets (numpy arrays) with the same number of samples
# X could be word embeddings, and Y could be molecular fingerprints, both already preprocessed

# Instantiate and fit the CCA model
cca = CCA(n_components=2)  # You can adjust 'n_components' based on your specific needs
cca.fit(X, Y)

# Transform the datasets
X_c, Y_c = cca.transform(X, Y)

# X_c and Y_c are the projections of X and Y onto the learned canonical variables
# You can now analyze the correlations or use these projections for further analysis
